In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Glioblastoma/GSE226976'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Combined oncolytic adenovirus DNX-2401 and anti-PD-checkpoint inhibitor pembrolizumab for recurrent glioblastoma"
!Series_summary	"Gene expression data for samples included in this trial of DNX2401 and pembrolizumab for recurrent glioma"
!Series_overall_design	"Observational cohort"
!Series_overall_design	"CAPTIVE investigators"
Sample Characteristics Dictionary:
{0: ['sample type: recurrent glioma']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0
age_row = gender_row = None  # no information available in given dictionary
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

def convert_trait(value):
    try:
        trait_value = value.split(":")[-1].strip()
        return 1 if "glioma" in trait_value.lower() else 0
    except Exception:
        return None

# Save cohort information
save_cohort_info('GSE226976', './preprocessed/Glioblastoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Glioblastoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Glioblastoma/trait_data/GSE226976.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM7089240': [1], 'GSM7089241': [1], 'GSM7089242': [1], 'GSM7089243': [1], 'GSM7089244': [1], 'GSM7089245': [1], 'GSM7089246': [1], 'GSM7089247': [1], 'GSM7089248': [1], 'GSM7089249': [1], 'GSM7089250': [1], 'GSM7089251': [1], 'GSM7089252': [1], 'GSM7089253': [1], 'GSM7089254': [1], 'GSM7089255': [1], 'GSM7089256': [1], 'GSM7089257': [1], 'GSM7089258': [1], 'GSM7089259': [1], 'GSM7089260': [1], 'GSM7089261': [1], 'GSM7089262': [1], 'GSM7089263': [1], 'GSM7089264': [1], 'GSM7089265': [1], 'GSM7089266': [1], 'GSM7089267': [1], 'GSM7089268': [1], 'GSM7089269': [1], 'GSM7089270': [1], 'GSM7089271': [1], 'GSM7089272': [1], 'GSM7089273': [1], 'GSM7089274': [1], 'GSM7089275': [1], 'GSM7089276': [1], 'GSM7089277': [1], 'GSM7089278': [1], 'GSM7089279': [1], 'GSM7089280': [1], 'GSM7089281': [1], 'GSM7089282': [1], 'GSM7089283': [1], 'GSM7089284': [1], 'GSM7089285': [1], 'GSM7089286': [1], 'GSM7089287': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1', 'ADM', 'ADORA2A', 'AKT1',
       'ALDOA', 'ALDOC', 'ANGPT1', 'ANGPT2', 'ANGPTL4', 'ANLN', 'APC', 'APH1B',
       'API5', 'APLNR', 'APOE', 'APOL6'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Since requires_gene_mapping is False, we skip STEP6


Gene annotation preview:
{'ID': ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1'], 'ORF': ['A2M', 'ABCF1', 'ACVR1C', 'ADAM12', 'ADGRE1']}


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Glioblastoma/gene_data/GSE226976.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Glioblastoma')

# 4. Save the cohort information.
json_path = './preprocessed/Glioblastoma/cohort_info.json'
save_cohort_info('GSE226976', json_path, True, True, trait_biased, unbiased_merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Glioblastoma/GSE226976.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Glioblastoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.

A new JSON file was created at: ./preprocessed/Glioblastoma/cohort_info.json
